In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
df = pd.read_csv('./../Data/Old_Bailey_speech_features.csv', index_col=0)
df.rename({'label': 'gold_label'}, axis=1, inplace=True)

In [7]:
df.head()

,filename,sent_id,speaker,text,gold_label,first_name,last_name,tokens,lemmas,pos,...,WordsVsUnique,Count_Adv_hedging,Count_V_hedging,Count_V_conj_hedging,Count_Adj_hedging,Count_Adv_intensifying,Count_V_intensifying,Count_Adj_intensifying,Count_Yes_no,gender
0,1800-01-15_t18000115-10-verdict52.txt,0,JOHN PARTRIDGE,"- I live in Craven-street, in the Strand, and...",2,JOHN,PARTRIDGE,"['-', 'I', 'live', 'in', 'Craven', '-', 'stree...","['-', '-PRON-', 'live', 'in', 'Craven', '-', '...","[97, 95, 100, 85, 96, 97, 92, 97, 85, 90, 96, ...",...,0.833333,0,0,0,0,0,0,0,0,male
1,1800-01-15_t18000115-10-verdict52.txt,1,JOHN PARTRIDGE,"On the 23d of November, I was robbed about a q...",1,JOHN,PARTRIDGE,"['On', 'the', '23d', 'of', 'November', ',', 'I...","['on', 'the', '23d', 'of', 'November', ',', '-...","[85, 90, 92, 85, 96, 97, 95, 100, 100, 85, 90,...",...,0.880000,1,1,0,1,0,0,0,0,male
2,1800-01-15_t18000115-10-verdict52.txt,2,JOHN PARTRIDGE,"No, it was not; it was a clear evening; a man ...",3,JOHN,PARTRIDGE,"['No', ',', 'it', 'was', 'not', ';', 'it', 'wa...","['no', ',', '-PRON-', 'be', 'not', ';', '-PRON...","[91, 97, 95, 100, 86, 97, 95, 100, 90, 84, 92,...",...,0.800000,0,0,0,0,0,0,1,1,male
3,1800-01-15_t18000115-10-verdict52.txt,3,JOHN PARTRIDGE,It struck me very forcibly the moment I saw hi...,3,JOHN,PARTRIDGE,"['It', 'struck', 'me', 'very', 'forcibly', 'th...","['-PRON-', 'strike', '-PRON-', 'very', 'forcib...","[95, 100, 95, 86, 86, 90, 92, 95, 100, 95, 97]",...,1.000000,0,0,0,0,2,0,0,0,male
4,1800-01-15_t18000115-10-verdict52.txt,4,JOHN PARTRIDGE,I really have no doubt at all of it.,3,JOHN,PARTRIDGE,"['I', 'really', 'have', 'no', 'doubt', 'at', '...","['-PRON-', 'really', 'have', 'no', 'doubt', 'a...","[95, 86, 100, 90, 92, 86, 90, 85, 95, 97]",...,1.000000,0,0,0,0,1,0,1,1,male


In [8]:
df.columns

Index(['filename', 'sent_id', 'speaker', 'text', 'gold_label', 'first_name',
       'last_name', 'tokens', 'lemmas', 'pos', 'adv_count',
       'NumberOfCharacters', 'MeanCharactersPerWord', 'SentenceLength',
       'NumberofUniqueWords', 'WordsVsUnique', 'Count_Adv_hedging',
       'Count_V_hedging', 'Count_V_conj_hedging', 'Count_Adj_hedging',
       'Count_Adv_intensifying', 'Count_V_intensifying',
       'Count_Adj_intensifying', 'Count_Yes_no', 'gender'],
      dtype='object')

In [9]:
def hegd_intens_count(row):
    # First find powerless
    if any([row['Count_Adv_hedging'] > 0,
            row['Count_V_hedging'] > 0,
            row['Count_V_conj_hedging'] > 0,
            row['Count_Adj_hedging'] > 0]):
        value = 1
    # Then find powerful
    elif any([row['Count_Adv_intensifying'] > 0,
             row['Count_V_intensifying'] > 0,
             row['Count_Adj_intensifying'] > 0]):
        value = 3
    # Otherwise neutral
    else:
        value = 2
    return value

In [3]:
clf = SVC(class_weight='balanced')
clf

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [11]:
clf = SVC(class_weight='balanced')
clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [106]:
y_pred = clf.predict(X_test)

In [107]:
df_test['SVC_pred'] = y_pred

df_test['lexicon_label'] = df_test.apply(lambda row: hegd_intens_count(row), axis=1)

In [108]:
df_test

,filename,sent_id,speaker,text,gold_label,first_name,last_name,tokens,lemmas,pos,...,Count_Adv_hedging,Count_V_hedging,Count_V_conj_hedging,Count_Adj_hedging,Count_Adv_intensifying,Count_V_intensifying,Count_Adj_intensifying,Count_Yes_no,SVC_pred,lexicon_label
1860,1800-02-19_t18000219-74-verdict451.txt,34,SUSANNA BOSWELL,Certainly; it was a very complicated examinati...,3,SUSANNA,BOSWELL,"['Certainly', ';', 'it', 'was', 'a', 'very', '...","['certainly', ';', '-PRON-', 'be', 'a', 'very'...","[86, 97, 95, 100, 90, 86, 84, 92, 97, 86, 100,...",...,1,1,0,0,1,1,1,0,1,1
353,1800-01-15_t18000115-25-punish136.txt,8,SARAH PAUL,No.,2,SARAH,PAUL,"['No', '.']","['no', '.']","[91, 97]",...,0,0,0,0,0,0,0,1,2,2
1333,1800-02-19_t18000219-10-punish56.txt,101,WILLIAM KNIGHT,"yes; then Knight said, was there no quarrel, h...",2,WILLIAM,KNIGHT,"['yes', ';', 'then', 'Knight', 'said', ',', 'w...","['yes', ';', 'then', 'Knight', 'say', ',', 'be...","[91, 97, 86, 96, 100, 97, 100, 86, 90, 92, 97,...",...,1,0,1,0,1,0,0,4,1,1
905,1800-01-15_t18000115-85-verdict507.txt,96,JOHN PHILPOT,Yes.,2,JOHN,PHILPOT,"['Yes', '.']","['yes', '.']","[91, 97]",...,0,0,0,0,0,0,0,1,2,2
1289,1800-02-19_t18000219-10-punish56.txt,53,WILLIAM DAVIS,A private.,2,WILLIAM,DAVIS,"['A', 'private', '.']","['a', 'private', '.']","[90, 84, 97]",...,0,0,0,0,0,0,0,0,2,2
1273,1800-02-19_t18000219-10-punish56.txt,37,BENJAMIN RUDKIN,I never did.,3,BENJAMIN,RUDKIN,"['I', 'never', 'did', '.']","['-PRON-', 'never', 'do', '.']","[95, 86, 100, 97]",...,0,0,0,0,1,0,0,0,3,3
938,1800-01-15_t18000115-85-verdict507.txt,133,EDWARD WALLIS,Yes; I have often seen him write.,1,EDWARD,WALLIS,"['Yes', ';', 'I', 'have', 'often', 'seen', 'hi...","['yes', ';', '-PRON-', 'have', 'often', 'see',...","[91, 97, 95, 100, 86, 100, 95, 100, 97]",...,1,0,0,0,0,0,0,1,1,1
1731,1800-02-19_t18000219-67-verdict412.txt,9,JAMES IRELAND,Yes.,2,JAMES,IRELAND,"['Yes', '.']","['yes', '.']","[91, 97]",...,0,0,0,0,0,0,0,1,2,2
65,1800-01-15_t18000115-10-verdict52.txt,68,JAMES RIBBENS,"No, he did not.",2,JAMES,RIBBENS,"['No', ',', 'he', 'did', 'not', '.']","['no', ',', '-PRON-', 'do', 'not', '.']","[91, 97, 95, 100, 86, 97]",...,0,0,0,0,0,0,0,1,2,2
1323,1800-02-19_t18000219-10-punish56.txt,88,THOMAS PEAKE,"He sat in the box, and he drew towards the end...",2,THOMAS,PEAKE,"['He', 'sat', 'in', 'the', 'box', ',', 'and', ...","['-PRON-', 'sit', 'in', 'the', 'box', ',', 'an...","[95, 100, 85, 90, 92, 97, 89, 95, 100, 85, 90,...",...,0,0,0,0,0,0,0,0,2,2


In [109]:
target_names = ['Powerless', 'Neutral', 'Powerful']
print('SVC_pred')
print(classification_report(df_test['gold_label'], df_test['SVC_pred'], target_names=target_names))

SVC_pred
              precision    recall  f1-score   support

   Powerless       0.39      0.79      0.52       125
     Neutral       0.91      0.66      0.77       468
    Powerful       0.44      0.42      0.43        67

    accuracy                           0.66       660
   macro avg       0.58      0.62      0.57       660
weighted avg       0.76      0.66      0.69       660



In [110]:
print('lexicon_label')
print(classification_report(df_test['gold_label'], df_test['lexicon_label'], target_names=target_names))

lexicon_label
              precision    recall  f1-score   support

   Powerless       0.49      0.62      0.55       125
     Neutral       0.87      0.78      0.83       468
    Powerful       0.45      0.57      0.50        67

    accuracy                           0.73       660
   macro avg       0.61      0.66      0.62       660
weighted avg       0.76      0.73      0.74       660

